In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import TruncatedSVD
from sklearn import preprocessing
from gensim.models import Word2Vec
import seaborn as sns
import jieba
import re
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

### 01.读取数据

In [ ]:
import pandas as pd
hole_df = pd.read_csv("../data/2023-06-08_holes_day.csv", index_col=0)
hole_df.dropna(inplace=True)
hole_df.shape

In [ ]:
hole_df

In [ ]:
hole_df['reply'] = hole_df.reply.astype(float)
hole_df['likenum'] = hole_df.likenum.astype(float)

In [ ]:
hole_df['hot'] = hole_df.likenum*hole_df.reply

In [ ]:
hole_df.sort_values('hot').tail(20)

In [ ]:
hole_df.loc[:,'hour'] = hole_df.time.apply(lambda x: x.split()[1].split(':')[0]).astype(int)

In [ ]:
hole_df.groupby('hour').count().text

In [ ]:
sns.set()
sns.lineplot(hole_df.groupby('hour').count().text)

In [ ]:
sns.lineplot(hole_df.groupby('hour').hot.mean())

In [ ]:
# hole_df = hole_df[hole_df['hot'] != 0]

In [ ]:
from tqdm import tqdm
tqdm.pandas()

#由于筛出来太多无意义停止词，所以加入停止词表 https://raw.githubusercontent.com/goto456/stopwords/master/baidu_stopwords.txt
with open("baidu_stopwords.txt") as f:
    stop_word_list = f.read().splitlines()
useless_str = '''一！“”，。？、；’"',.、·《》（）()#\t：\n\r\n/'''

words = hole_df['text'].progress_apply(lambda x: ' '.join([x for x in jieba.cut(x) if x not in stop_word_list and x not in useless_str]))
hole_df['raw_text'] = hole_df['text'].copy()
hole_df['text'] = words

In [ ]:
# filter
# hole_df = hole_df.loc[(hole_df.hour<=2) | (hole_df.hour>=23),:]

In [ ]:
hole_df.loc[:,'hot_norm'] = hole_df.hot.apply(lambda x: x if x==0 else np.log(x))
hole_df.loc[:,'hot_norm'] = hole_df.hot_norm.apply(lambda x: x if x!=0 else 0.1)

In [ ]:
hot_hole = hole_df[hole_df.hot_norm>1]
hot_hole.shape

In [ ]:
tfi_hole_adj = pd.concat([hole_df, hot_hole, hot_hole], ignore_index=True)

In [ ]:
tfidf = TfidfVectorizer()
tfidf_vec = tfidf.fit_transform(tfi_hole_adj['text'])

In [ ]:
# Get the feature names from the vectorizer
feature_names = tfidf.get_feature_names_out()

# Get the sum of the tf-idf values for each feature across all documents
sum_tfidf = np.sum(tfidf_vec.toarray(), axis=0)

# Get the indices of the features sorted by their tf-idf sum in descending order
sorted_indices = np.argsort(sum_tfidf)[::-1]

# Print the top 10 features and their tf-idf sums
print("Top 10 features and their tf-idf sums")
for i in range(10):
    feature_index = sorted_indices[i]
    feature_name = feature_names[feature_index]
    tfidf_sum = sum_tfidf[feature_index]
    print(f"{feature_name}: {tfidf_sum}")

In [ ]:
# Word2Vec vectorization for segmented '简介'
sentences = [s.split() for s in hole_df['text']]

seed = 123
w2v = Word2Vec(sentences, vector_size=10, min_count=1, workers=4, seed=seed) # size is the dimensionality of the feature vectors

In [ ]:
for e in w2v.wv.most_similar(positive=['食堂'], topn=10):
   print(e[0], e[1])

In [ ]:
# We first tokenize each sentence in the storyline column
tokenized_sentences = hole_df.text.apply(lambda x: x.split())

# We then calculate the word vectors for each word in each sentence
word_vectors = tokenized_sentences.apply(lambda x: np.array([w2v.wv[word] for word in x if word in w2v.wv.key_to_index]))

# We then calculate the average vector for each sentence
sentence_vectors = word_vectors.apply(lambda x: np.mean(x, axis=0)) 

# The resulting sentence_vectors will contain the vector representation for each sentence in the storyline column.
sentence_vectors

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
vectorizer.get_feature_names_out()


X.toarray()

In [ ]:
w2v_sentence_vectors = np.stack(np.array(sentence_vectors))

In [ ]:
w2v_sentence_vectors.shape

In [ ]:
hole_df

In [ ]:
hole_df.loc[:,'hour_6am'] = hole_df.hour.apply(lambda x: x-6 if x-6 >= 0 else x-6+24)

In [ ]:
train_data = np.array(hole_df[['reply', 'likenum', 'hour_6am']])
train_data = np.hstack([train_data, w2v_sentence_vectors])
train_data.shape

In [ ]:
scaler = MinMaxScaler()
train_data = scaler.fit_transform(train_data)
pca_train = PCA(n_components=0.95)
train_data_pca = pca_train.fit_transform(train_data)

In [ ]:
# plot the scree plot
plt.plot(range(1, pca_train.n_components_ + 1), pca_train.explained_variance_ratio_.cumsum(), marker='o')
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance')
plt.show()

In [ ]:
# Task 3: K-means clustering

# Elbow method to determine optimal number of clusters
# Calculate the sum of squared errors for different values of k
# Choose the value of k at the "elbow" of the plot
# This is the point of diminishing returns, where adding more clusters doesn't significantly improve the SSE
# Set the value of k to the optimal number of clusters

from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Calculate SSE for different values of k
sse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
    kmeans.fit(train_data_pca)
    sse.append(kmeans.inertia_)

# Plot SSE against number of clusters
plt.plot(range(1, 11), sse)
plt.title('Elbow Method')
plt.xlabel('Number of Clusters')
plt.ylabel('SSE')
plt.show()

# Set the value of k to the optimal number of clusters
k = 30

# Perform K-means clustering with optimal number of clusters
kmeans = KMeans(n_clusters=k, random_state=114514, n_init='auto')
kmeans.fit_transform(train_data_pca)
# Use the trained KMeans model to predict the cluster labels for the test data
labels = kmeans.labels_


In [ ]:
# Create a dictionary to store the holes in each cluster
cluster_holes = {}
for i in range(k):
    cluster_holes[i] = []

# Iterate through the test data and add each movie to its corresponding cluster
for i in range(len(train_data_pca)):
    cluster_holes[labels[i]].append(hole_df.iloc[i])

# Output representative movies, number of movies in each cluster, and rating distribution
cluster_avg_hour_6am = []
cluster_avg_like = []
cluster_avg_reply = []
for i in range(k):
    print("====================")
    print(f"Cluster {i+1}:")
    print(f"Number of holes in cluster: {len(cluster_holes[i])}")
    print(f"Avg time(start at 6am) {hole_df[labels == i]['hour_6am'].mean()}")
    cluster_avg_hour_6am.append(hole_df[labels == i]['hour_6am'].mean())
    print(f"Like mean: {hole_df[labels == i]['likenum'].mean()}")
    cluster_avg_like.append(hole_df[labels == i]['likenum'].mean())
    print(f"Reply mean: {hole_df[labels == i]['reply'].mean()}")
    cluster_avg_reply.append(hole_df[labels == i]['reply'].mean())
    # Print the tags for each movie in the cluster
    print(f"Text for hole: {hole_df[labels == i]['raw_text'].sample(10)}")

    print("\n")

In [1]:
import pandas as pd
import sqlite3
import re

In [2]:
db_path = '/datb1/wuyuxuan/tmp/Holemonitor/data/2024-03-09_holes_monitor.db'
with sqlite3.connect(db_path) as conn:
    data = pd.read_sql("SELECT * FROM holes", conn)

In [3]:
data['reply'] = data.reply.astype(float)
data['likenum'] = data.likenum.astype(float)
data.loc[:,'hot'] = data.reply*data.likenum
data.sort_values(by='hot', ascending=False)

,pid,text,type,time,reply,likenum,last_retrive,hot
1068,6037913,popi 请问问题（以及dz可能回问）,text,2024-03-10 01:01:55,92.0,8.0,2024-03-10 01:27:01,736.0
1126,6037971,popi 👩 随便聊聊,text,2024-03-10 01:32:15,99.0,7.0,2024-03-10 02:20:22,693.0
488,6037332,洞主从22年12月“第一责任人”开始，因为家属认为新冠的后果还是要观察一段时间再下结论，就一...,text,2024-03-09 22:05:02,33.0,14.0,2024-03-09 22:23:19,462.0
309,6037153,树洞的朋友们大家好。有一件事情想向大家求助，dz现在得了很严重的病，很难治好的那种。很多年前...,text,2024-03-09 21:02:52,30.0,12.0,2024-03-09 21:19:39,360.0
1030,6037875,我很不喜欢北京。它现代，工整，四平八稳，过分地宽敞，显出一种冷漠来。冬春时是最讨厌的季节，刮...,text,2024-03-10 00:44:22,12.0,28.0,2024-03-10 01:06:35,336.0
...,...,...,...,...,...,...,...,...
577,6037421,原价20出一张周三晚上驾驶我的车 二楼中间靠后,text,2024-03-09 22:33:37,0.0,1.0,2024-03-09 22:46:29,0.0
580,6037424,6036526捞捞 闲置 二手 出衣服 多买可刀,text,2024-03-09 22:35:29,0.0,0.0,2024-03-09 22:49:11,0.0
583,6037427,收留可爱妹妹～,text,2024-03-09 22:36:20,0.0,1.0,2024-03-09 22:50:31,0.0
587,6037431,有点小痛苦,text,2024-03-09 22:36:58,0.0,1.0,2024-03-09 22:50:31,0.0


In [7]:
db_path = '/datb1/wuyuxuan/tmp/Holemonitor/data/2024-03-09_holes_monitor.db'
with sqlite3.connect(db_path) as conn:
    data_com = pd.read_sql("SELECT * FROM comments", conn)

In [23]:
data[data.pid.isin(data_com[data_com.text.str.contains('fwb')].pid)].sort_values(by='time', ascending=False)

,pid,text,type,time,reply,likenum,last_retrive,hot
755,6037599,被nanpy甩了，难受，开个洞popi😭,text,2024-03-09 23:25:56,28.0,6.0,2024-03-09 23:37:57,168.0


In [14]:
s = """
6036940 deleted!
6036940 deleted!
6036940 deleted!
6036940 deleted!
6036940 deleted!
6036940 deleted!
6036940 deleted!
6036940 deleted!
6036940 deleted!
6036980 deleted!
6036940 deleted!
6037025 deleted!
6037171 deleted!
6037180 deleted!
6037258 deleted!
6037367 deleted!
6037345 deleted!
6037812 deleted!
6037876 deleted!
6037921 deleted!
6037937 deleted!
"""
deleted = re.findall(r'(\d+) deleted!', s)
deleted = [int(x) for x in deleted]

In [15]:
data[data.pid.isin(deleted)]

,pid,text,type,time,reply,likenum,last_retrive,hot
99,6036940,聊聊最疯的经历？,text,2024-03-09 19:44:25,1.0,1.0,2024-03-09 20:05:31,1.0
137,6036980,蹲高一物理数学老师，地点大兴，时薪160。QQ6537️⃣7991️⃣8,text,2024-03-09 20:02:20,2.0,2.0,2024-03-09 20:04:01,4.0
182,6037025,兼职 家教,text,2024-03-09 20:14:49,4.0,3.0,2024-03-09 20:16:44,12.0
327,6037171,None,image,2024-03-09 21:07:59,1.0,2.0,2024-03-09 21:14:10,2.0
336,6037180,这边有个高三数学的单，是那种线上讲解题目，按题给钱的。有同学想接吗，顺便可以说一下价格。,text,2024-03-09 21:09:53,17.0,6.0,2024-03-09 21:16:50,102.0
414,6037258,有木有uu认识的外校的uu愿意做助教的，线下改初中作业题，200/天,text,2024-03-09 21:45:14,2.0,3.0,2024-03-09 21:49:00,6.0
501,6037345,大四小语种\n选一线城市公务员\n还是海硕一年\n（目前情况是，收到了港中文/港大和南洋理工...,text,2024-03-09 22:11:34,2.0,4.0,2024-03-09 22:21:54,8.0
523,6037367,昨天晚上让npy一夜cum了三次,text,2024-03-09 22:19:03,2.0,3.0,2024-03-09 22:20:23,6.0
967,6037812,想wen,text,2024-03-10 00:19:09,4.0,3.0,2024-03-10 00:24:48,12.0
1031,6037876,有没有直男想弯一下 找我,text,2024-03-10 00:44:46,1.0,2.0,2024-03-10 00:49:04,2.0


In [ ]:
from tqdm import tqdm
tqdm.pandas()

#由于筛出来太多无意义停止词，所以加入停止词表 https://raw.githubusercontent.com/goto456/stopwords/master/baidu_stopwords.txt
with open("baidu_stopwords.txt") as f:
    stop_word_list = f.read().splitlines()
useless_str = '''一！“”，。？、；’"',.、·《》（）()#\t：\n\r\n/'''

words = data['text'].progress_apply(lambda x: ' '.join([x for x in jieba.cut(x) if x not in stop_word_list and x not in useless_str]))
data['raw_text'] = data['text'].copy()
data['text'] = words

In [ ]:
data

In [ ]:
tfidf = TfidfVectorizer()
tfidf_vec = tfidf.fit_transform(data['text'])

# Get the feature names from the vectorizer
feature_names = tfidf.get_feature_names_out()

# Get the sum of the tf-idf values for each feature across all documents
sum_tfidf = np.sum(tfidf_vec.toarray(), axis=0)

# Get the indices of the features sorted by their tf-idf sum in descending order
sorted_indices = np.argsort(sum_tfidf)[::-1]

# Print the top 10 features and their tf-idf sums
print("Top 10 features and their tf-idf sums")
for i in range(10):
    feature_index = sorted_indices[i]
    feature_name = feature_names[feature_index]
    tfidf_sum = sum_tfidf[feature_index]
    print(f"{feature_name}: {tfidf_sum}")

In [ ]:
# Word2Vec vectorization for segmented '简介'
sentences = [s.split() for s in data['text']]

seed = 123
w2v = Word2Vec(sentences, vector_size=10, min_count=1, workers=4, seed=seed) # size is the dimensionality of the feature vectors

In [ ]:
db_path = '../data/2023-06-14_holes_monitor.db'
with sqlite3.connect(db_path) as conn:
    data_holes = pd.read_sql("SELECT * FROM holes", conn)
    data_comments = pd.read_sql("SELECT * FROM comments", conn)

data_holes.to_csv("../data/614holes.csv", index=True)
data_comments.to_csv("../data/614comments.csv", index=True)

In [ ]:
import re

In [ ]:
db_path = '../data/2023-06-19_holes_monitor.db'
with sqlite3.connect(db_path) as conn:
    data = pd.read_sql("SELECT * FROM holes", conn)

In [ ]:
kw = ".*(短租|租房).*"

In [ ]:
data[data.text.str.match(kw, flags=re.DOTALL)]